In [28]:
from keras.layers import Input, Dense
from keras.layers.core import Reshape
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.layers.convolutional import MaxPooling2D,UpSampling2D,Conv2DTranspose
from keras.layers.convolutional import Convolution2D as Conv2D
from keras.layers.normalization import BatchNormalization
from keras.layers import merge
from keras.optimizers import Adadelta, RMSprop
import os
import os.path
import numpy as np
from PIL import Image
from numpy import * 
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
import scipy.misc
import cv2

In [29]:
def model():

    input_img = Input(shape=(128,128,1))
    conv1 = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
      
    conv2 = Conv2D(32, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
       
    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    
    up6 = UpSampling2D((2,2))(conv5)
    up6 = merge([up6, conv4], mode='concat', concat_axis=3)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(up6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    
    up7 = UpSampling2D((2,2))(conv6)
    up7 = merge([up7, conv3], mode='concat', concat_axis=3)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(up7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    
    up8 = UpSampling2D((2,2))(conv7)
    up8 = merge([up8, conv2], mode='concat', concat_axis=3)
    conv8 = Conv2D(32, (3, 3), activation='relu', padding='same')(up8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv8)
    conv8 = BatchNormalization()(conv8)
    
    up9 = UpSampling2D((2,2))(conv8)
    up9 = merge([up9, conv1], mode='concat', concat_axis=3)
    conv9 = Conv2D(16, (3, 3), activation='relu', padding='same')(up9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv9)
    conv9 = BatchNormalization()(conv9)
       
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(conv9)
    
    ada=Adadelta(lr=5.0, rho=0.95, epsilon=1e-08, decay=0.001)
    rms=RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.001)
    
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(loss='mean_squared_error', optimizer=rms)
    return autoencoder

In [30]:
autoencoder=model()

basic_mat=[]
tobe_mat=[]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:34: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:41: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:48: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:55: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [31]:
path1="Data_M"

Loading Data, Converting it from RGB to Grayscale

In [32]:
for i in range(1,123):
    path_major=path1+'/'+str(i)
    for j in range(1,100):
        img=array(Image.open(path_major+"/"+str(j)+"_.jpg"))
        #print shape(img)
        img = cv2.cvtColor( img, cv2.COLOR_RGB2GRAY )
        img=img.reshape(128,128,1)
        basic_mat.append(img)
        tobe_mat.append(img)

In [33]:
data,Label = shuffle(basic_mat,tobe_mat, random_state=2)

Spliting the Train and Test Data

In [34]:
X_train, X_test, y_train, y_test = train_test_split(data, Label, test_size=0.2, random_state=2)
X_train = array(X_train)
y_train = array(y_train)
X_test = array(X_test)
y_test = array(y_test)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(878, 128, 128, 1)
(878, 128, 128, 1)
(220, 128, 128, 1)
(220, 128, 128, 1)


Normalising the Values between 0 and 1

In [35]:
x_train = X_train.astype('float32') / 255.
x_test = X_test.astype('float32') / 255.

y_train = y_train.astype('float32') / 255.
y_test = y_test.astype('float32') / 255.
CheckDir = 'sample/'

In [36]:
for epoch in range(1,300):
    
    train_X,train_Y=shuffle(x_train,y_train)
    print ("Epoch is: %d\n" % epoch)
    batch_size=64
    print ("Number of batches: %d\n" % int(len(train_X)/batch_size))
    num_batches=int(len(train_X)/batch_size)
    for batch in range(num_batches):    
        batch_train_X=train_X[batch*batch_size:min((batch+1)*batch_size,len(train_X))]
        batch_train_Y=train_Y[batch*batch_size:min((batch+1)*batch_size,len(train_Y))]
        loss=autoencoder.train_on_batch(batch_train_X,batch_train_Y)
        print ('epoch_num: %d batch_num: %d loss: %f\n' % (epoch,batch,loss))

    autoencoder.save_weights("fabric_autoen.h5")
    if(epoch%2==0):
        x_test,y_test=shuffle(x_test,y_test)
        decoded_imgs=autoencoder.predict(x_test[:2])
        temp = np.zeros([128, 128*3,3])
        temp[:, :128,:1] = x_test[0,:,:,:1]
        temp[:, 128:128*2,:1] = y_test[0,:,:,:1]
        temp[:, 128*2:,:1] = decoded_imgs[0,:,:,:1]
        temp[:,:,1]=temp[:,:,0]
        temp[:,:,2]=temp[:,:,0]
        temp = temp*255
        scipy.misc.imsave(CheckDir + str(epoch) + ".jpg", temp)

Epoch is: 1

Number of batches: 6

epoch_num: 1 batch_num: 0 loss: 0.126357

epoch_num: 1 batch_num: 1 loss: 0.045475

epoch_num: 1 batch_num: 2 loss: 0.036353

epoch_num: 1 batch_num: 3 loss: 0.020744

epoch_num: 1 batch_num: 4 loss: 0.017121

epoch_num: 1 batch_num: 5 loss: 0.014984

Epoch is: 2

Number of batches: 6

epoch_num: 2 batch_num: 0 loss: 0.011565

epoch_num: 2 batch_num: 1 loss: 0.009701

epoch_num: 2 batch_num: 2 loss: 0.007675

epoch_num: 2 batch_num: 3 loss: 0.006582

epoch_num: 2 batch_num: 4 loss: 0.005638

epoch_num: 2 batch_num: 5 loss: 0.005281

Epoch is: 3

Number of batches: 6

epoch_num: 3 batch_num: 0 loss: 0.004274

epoch_num: 3 batch_num: 1 loss: 0.005603

epoch_num: 3 batch_num: 2 loss: 0.004505

epoch_num: 3 batch_num: 3 loss: 0.004315

epoch_num: 3 batch_num: 4 loss: 0.003987

epoch_num: 3 batch_num: 5 loss: 0.004229

Epoch is: 4

Number of batches: 6

epoch_num: 4 batch_num: 0 loss: 0.003600

epoch_num: 4 batch_num: 1 loss: 0.003876

epoch_num: 4 batch_n

epoch_num: 29 batch_num: 0 loss: 0.000447

epoch_num: 29 batch_num: 1 loss: 0.000541

epoch_num: 29 batch_num: 2 loss: 0.000691

epoch_num: 29 batch_num: 3 loss: 0.000687

epoch_num: 29 batch_num: 4 loss: 0.000690

epoch_num: 29 batch_num: 5 loss: 0.000745

Epoch is: 30

Number of batches: 6

epoch_num: 30 batch_num: 0 loss: 0.000786

epoch_num: 30 batch_num: 1 loss: 0.000777

epoch_num: 30 batch_num: 2 loss: 0.000984

epoch_num: 30 batch_num: 3 loss: 0.000604

epoch_num: 30 batch_num: 4 loss: 0.000658

epoch_num: 30 batch_num: 5 loss: 0.000780

Epoch is: 31

Number of batches: 6

epoch_num: 31 batch_num: 0 loss: 0.000656

epoch_num: 31 batch_num: 1 loss: 0.000757

epoch_num: 31 batch_num: 2 loss: 0.000556

epoch_num: 31 batch_num: 3 loss: 0.000473

epoch_num: 31 batch_num: 4 loss: 0.000341

epoch_num: 31 batch_num: 5 loss: 0.000805

Epoch is: 32

Number of batches: 6

epoch_num: 32 batch_num: 0 loss: 0.001043

epoch_num: 32 batch_num: 1 loss: 0.001562

epoch_num: 32 batch_num: 2 loss:

epoch_num: 57 batch_num: 0 loss: 0.000480

epoch_num: 57 batch_num: 1 loss: 0.000232

epoch_num: 57 batch_num: 2 loss: 0.000250

epoch_num: 57 batch_num: 3 loss: 0.000307

epoch_num: 57 batch_num: 4 loss: 0.000747

epoch_num: 57 batch_num: 5 loss: 0.000530

Epoch is: 58

Number of batches: 6

epoch_num: 58 batch_num: 0 loss: 0.000376

epoch_num: 58 batch_num: 1 loss: 0.000454

epoch_num: 58 batch_num: 2 loss: 0.000570

epoch_num: 58 batch_num: 3 loss: 0.000551

epoch_num: 58 batch_num: 4 loss: 0.000597

epoch_num: 58 batch_num: 5 loss: 0.000356

Epoch is: 59

Number of batches: 6

epoch_num: 59 batch_num: 0 loss: 0.000329

epoch_num: 59 batch_num: 1 loss: 0.000269

epoch_num: 59 batch_num: 2 loss: 0.000259

epoch_num: 59 batch_num: 3 loss: 0.000342

epoch_num: 59 batch_num: 4 loss: 0.000383

epoch_num: 59 batch_num: 5 loss: 0.000459

Epoch is: 60

Number of batches: 6

epoch_num: 60 batch_num: 0 loss: 0.000320

epoch_num: 60 batch_num: 1 loss: 0.000216

epoch_num: 60 batch_num: 2 loss:

epoch_num: 85 batch_num: 0 loss: 0.000265

epoch_num: 85 batch_num: 1 loss: 0.000685

epoch_num: 85 batch_num: 2 loss: 0.000511

epoch_num: 85 batch_num: 3 loss: 0.000185

epoch_num: 85 batch_num: 4 loss: 0.000285

epoch_num: 85 batch_num: 5 loss: 0.000226

Epoch is: 86

Number of batches: 6

epoch_num: 86 batch_num: 0 loss: 0.000532

epoch_num: 86 batch_num: 1 loss: 0.000292

epoch_num: 86 batch_num: 2 loss: 0.000362

epoch_num: 86 batch_num: 3 loss: 0.000481

epoch_num: 86 batch_num: 4 loss: 0.000362

epoch_num: 86 batch_num: 5 loss: 0.000232

Epoch is: 87

Number of batches: 6

epoch_num: 87 batch_num: 0 loss: 0.000583

epoch_num: 87 batch_num: 1 loss: 0.000258

epoch_num: 87 batch_num: 2 loss: 0.000189

epoch_num: 87 batch_num: 3 loss: 0.000373

epoch_num: 87 batch_num: 4 loss: 0.000233

epoch_num: 87 batch_num: 5 loss: 0.000202

Epoch is: 88

Number of batches: 6

epoch_num: 88 batch_num: 0 loss: 0.000350

epoch_num: 88 batch_num: 1 loss: 0.000313

epoch_num: 88 batch_num: 2 loss: